In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression, LogisticRegression
from IPython.display import clear_output
from datetime import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import json
import numpy as np
import scipy as sp
import scipy.stats as st
import scipy.integrate as integrate
from sklearn import linear_model
from scipy.stats import multivariate_normal
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import statsmodels.api as sm

from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import csr_matrix, hstack

import pickle, json
from scipy.stats import spearmanr, kendalltau

from scipy.optimize import fmin_tnc

import warnings
warnings.filterwarnings("ignore")

C:\Users\artys\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
%%time
with open("chgk/tournaments.pkl", "rb") as f:
    tournaments = pickle.load(f)
with open("chgk/results.pkl", "rb") as f:
    results = pickle.load(f)
with open("chgk/players.pkl", "rb") as f:
    players = pickle.load(f)

Wall time: 16.4 s


# Задание 1

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
- взять в тренировочный набор турниры с dateStart из 2019 года; 
- в тестовый — турниры с dateStart из 2020 года.

## Решение
    Оставляем турниры, у которых есть информация о составах команд и есть информация об ответах команд на вопросы
    Вопросы с отметкой X удаляются из выборки, вопросы с отметкой ? меняются на 0

In [3]:
def has_teamMembers_and_mask(tournament):
    flg = True
    for team in tournament:
        if (len(team['teamMembers']) == 0) or (team.get('mask') is None):
            flg = False
    return flg

In [4]:
%%time
# поправка на наличие инфо о составе и ответах по вопросам
results_corrected  = [i for i in results.keys() if has_teamMembers_and_mask(results[i])]

Wall time: 296 ms


In [5]:
%%time
# Делим чемпионаты на train (2019 год) и test (2020 год)
train_tournaments_labels = []
test_tournaments_labels = []
for i in tournaments.keys():
    if i in results_corrected:
        if tournaments[i]['dateStart'][:4] == '2019':
            train_tournaments_labels.append(i)
        elif tournaments[i]['dateStart'][:4] == '2020':
            test_tournaments_labels.append(i)
            
print('Количество чемпионатов в TRAIN :', len(train_tournaments_labels))
print('Количество чемпионатов в TEST :', len(test_tournaments_labels))

Количество чемпионатов в TRAIN : 657
Количество чемпионатов в TEST : 386
Wall time: 663 ms


In [6]:
def to_int(x):
    try:
        return int(x)
    except:
        return 0

In [7]:
%%time
# Вопросы с отметкой X удаляются из выборки, вопросы с отметкой ? меняются на 0
data = {
    'tournament_id': [],
    'tournament_name': [],
    'team_id': [],
    'player_id': [],
    'player_name': [],
    'player_surname': [],
    'question_num': [],
    'question_result': []
}

for i in train_tournaments_labels:
    for team in results[i]:
        for player in team['teamMembers']:
            for question_num in range(len(team['mask'])):
                if team['mask'][question_num] != 'X':
                    data['tournament_id'].append(tournaments[i]['id'])
                    data['tournament_name'].append(tournaments[i]['name'])
                    data['team_id'].append(team['team']['id'])
                    data['player_id'].append(player['player']['id'])
                    data['player_name'].append(player['player']['name'])
                    data['player_surname'].append(player['player']['surname'])
                    data['question_num'].append(question_num)
                    data['question_result'].append(to_int(team['mask'][question_num]))
                
                
data_train = pd.DataFrame.from_dict(data)

Wall time: 54.1 s


# Задание 2

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
- повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
- в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
- для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


## Решение
    Соберем следующий набор признаков:
    - Доля команд, участвующая в турнире
    - Среднее количество ответов на турнире
    - Среднее количество ответов на вопрос на турнире
    - Среднее количество ответов игроком (в составе команды) за все время
    
    В качестве baseline модели будем использовать логистическую регерессию, обученную на одном атрибуте - среднее количество ответов на вопрос, среднее количество ответов одного игрока и среднее количество правильных ответов по всем игрокам
    
    Таблица рейтинга игроков формируется следующим образом: по каждому игроку считается сумма вероятностей ответа игроком на вопрос

In [8]:
data_train = data_train.merge((data_train.groupby('tournament_id')['team_id'].nunique()/data_train['team_id'].nunique()).to_frame('teams_qnt_rate').reset_index(), on='tournament_id', how='left')
data_train = data_train.merge(data_train.groupby('tournament_id')['question_result'].mean().to_frame('mean_tournament_answered').reset_index(), on='tournament_id', how='left')
data_train = data_train.merge(data_train.groupby(['tournament_id', 'question_num'])['question_result'].mean().to_frame('question_dificulty').reset_index(), on=['tournament_id', 'question_num'], how='left')
data_train = data_train.merge(data_train.groupby(['tournament_id', 'player_id'])['question_result'].mean().to_frame('player_strength').reset_index(), on=['tournament_id', 'player_id'], how='left')

In [9]:
data_train.head()

,tournament_id,tournament_name,team_id,player_id,player_name,player_surname,question_num,question_result,teams_qnt_rate,mean_tournament_answered,question_dificulty,player_strength
0,4772,Синхрон северных стран. Зимний выпуск,45556,6212,Юрий,Выменец,0,1,0.021079,0.475605,0.892295,0.777778
1,4772,Синхрон северных стран. Зимний выпуск,45556,6212,Юрий,Выменец,1,1,0.021079,0.475605,0.776305,0.777778
2,4772,Синхрон северных стран. Зимний выпуск,45556,6212,Юрий,Выменец,2,1,0.021079,0.475605,0.463132,0.777778
3,4772,Синхрон северных стран. Зимний выпуск,45556,6212,Юрий,Выменец,3,1,0.021079,0.475605,0.541839,0.777778
4,4772,Синхрон северных стран. Зимний выпуск,45556,6212,Юрий,Выменец,4,1,0.021079,0.475605,0.888981,0.777778


In [10]:
# Формируем выборки для обучения модели
X_train = (data_train['question_dificulty'] + data_train['player_strength'] + data_train['question_result'].mean()).values
X_train = X_train.reshape(-1, 1)
X_train = np.append(X_train, np.ones((X_train.shape[0], 1)), axis=1)
y_train = data_train['question_result'].values
y_train = y_train.reshape(-1, 1)

In [11]:
%%time
# Обучаем модель
model = LogisticRegression(solver = 'lbfgs')
model.fit(X_train[:, 0].reshape(-1, 1), y_train)

Wall time: 19 s


LogisticRegression()

In [12]:
# Функция для формирования рейтинга игроков
def calc_rating(prediction, player_rating_table=None, rating_name='rating'):
    
    temp_player_rating = data_train['player_id'].to_frame().copy()
    temp_player_rating['probability'] = prediction
    temp_player_rating = temp_player_rating.groupby('player_id')['probability'].sum().to_frame(rating_name).reset_index()
    
    if player_rating_table is None:
        player_rating_table = temp_player_rating
    else:
        if rating_name in player_rating_table.columns:
            player_rating_table[rating_name] = temp_player_rating[rating_name]
        else:
            player_rating_table = player_rating_table.merge(temp_player_rating, how='left', on='player_id')
        
    return player_rating_table

Топ самых сильных игроков

In [13]:
player_rating = calc_rating(model.predict_proba(X_train[:, 0].reshape(-1, 1))[:, 1], player_rating_table=None)
player_rating.merge(data_train[['player_id', 'player_name', 'player_surname']].drop_duplicates(), on='player_id', how='left').sort_values('rating', ascending=False).head(10)

,player_id,rating,player_name,player_surname
11749,110920,4511.166380,Алексей,Дворянчиков
14741,131908,4457.825563,Иван,Киселёв
2897,20640,4409.243184,Ольга,Мельникова
17507,147450,4383.779235,Максим,Салихов
1724,12511,4266.665701,Константин,Ильин
12688,117262,3971.888631,Сергей,Черкасов
3843,27499,3844.373831,Наталия,Рыжанова
3729,26640,3833.821710,Вадим,Раскумандрин
17965,149724,3816.944492,Дмитрий,Пелихов
9040,85059,3713.230393,Ева,Берг


# Задание 3

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.


## Решение
Рейтинг команды будем оценивать, как сумма рейтингов участников команды

In [14]:
%%time
# Обработка тестовой выборки - чемпионаты 2020
# Вопросы с отметкой X удаляются из выборки, вопросы с отметкой ? меняются на 0
data = {
    'tournament_id': [],
    'tournament_name': [],
    'team_id': [],
    'player_id': [],
    'position' : [],
}

for i in test_tournaments_labels:
    for team in results[i]:
        for player in team['teamMembers']:
            data['tournament_id'].append(tournaments[i]['id'])
            data['tournament_name'].append(tournaments[i]['name'])
            data['team_id'].append(team['team']['id'])
            data['player_id'].append(player['player']['id'])
            data['position'].append(int(team['position']))
                                
data_test = pd.DataFrame.from_dict(data)

Wall time: 763 ms


In [15]:
# Функция для проверки качества рейтингования игроков на тестовом наборе данных
def check_quality(test_df, rating_df, rating_name):
    
    test_df = test_df.merge(rating_df, on='player_id', how='left')
    test_df[rating_name] = test_df[rating_name].fillna(0)
    data_test_grouped = test_df.groupby(['tournament_id', 'team_id', 'position'])[rating_name].sum().to_frame().reset_index()

    spearmanr_list = []
    kendalltau_list = []

    for i in data_test_grouped['tournament_id'].drop_duplicates():
        temp = data_test_grouped[data_test_grouped['tournament_id']==i]
        if temp.shape[0] > 1:
            spearmanr_value, _ = spearmanr(temp['position'], temp[rating_name])
            kendalltau_value, _ = kendalltau(temp['position'], temp[rating_name])
            spearmanr_list.append(spearmanr_value)
            kendalltau_list.append(kendalltau_value)

    return np.mean(spearmanr_list), np.mean(kendalltau_list)

In [16]:
# Проверка качества: средняя корреляция Спирмена и Кендала
spearman, kendal = check_quality(test_df=data_test.copy(), 
                                 rating_df=player_rating.copy(), 
                                 rating_name='rating')
print('Корреляция Спирмана:', spearman)
print('Корреляция Кендала:', kendal)

Корреляция Спирмана: -0.6757990629387767
Корреляция Кендала: -0.5157142603613002


# Задание 4

Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
- предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
- разработайте EM-схему для обучения этой модели, реализуйте её в коде;
- обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


## Решение
    Основное предположение: 
    - если команда не ответила на вопрос, значит ни один из игроков не ответил на вопрос
    - если команда отвтеила на вопрос, значит хотя бы один игрок команды ответил на вопрос
    
    EM-алгоритм будет состоять в следующем:
    - введем скрытую переменную z - игрок ответил на вопрос
    - на M-шаге мы фиксируем текущие значения z для игроков и обучаем логистическую регрессию
    - на Е-шаге мы прогнозируем новые вероятности z для игроков, а также делаем корректировку этих вероятностей:
        - если команда не ответила на вопрос (y==0), то z = 0
        - если команда ответила на вопрос (у==1), то z = z / вероятность того, что хотя бы один игрок из команды ответил на этот вопрос

In [17]:
# Функции, реализующие логистическую регрессию, которая может обрабатывать нецелые значения в качестве целевого события
def sigmoid(x):
    # Activation function used to map any real value between 0 and 1
    return 1 / (1 + np.exp(-x))

def net_input(theta, x):
    # Computes the weighted sum of inputs
    return np.dot(x, theta)

def probability(theta, x):
    # Returns the probability after passing through sigmoid
    return sigmoid(net_input(theta, x))

def cost_function(theta, x, y):
    # Computes the cost function for all the training samples
    m = x.shape[0]
    total_cost = -(1 / m) * np.sum(
        y * np.log(probability(theta, x)) + (1 - y) * np.log(
            1 - probability(theta, x)))
    return total_cost

def gradient(theta, x, y):
    # Computes the gradient of the cost function at the point theta
    m = x.shape[0]
    return (1 / m) * np.dot(x.T, sigmoid(net_input(theta,   x)) - y)

def fit(x, y, theta):
    opt_weights = fmin_tnc(func=cost_function, x0=theta,
                  fprime=gradient,args=(x, y.flatten()))
    return opt_weights[0]

In [18]:
def em_step(X_train, y_train, z_train=None, write_rating_to=player_rating, rating_name='rating_new'):
    
    # только для первого шага
    if z_train is None:
        z_train = y_train.copy()
    
    # для всех остальных шагов, включая первый
    theta = np.zeros((X_train.shape[1], 1))
    parameters = fit(X_train, z_train, theta)
    
    parameters[1] = parameters[1] + np.log((pi * n_u) / (n_0 + pi * n_u))
    
    z_pred = probability(parameters, X_train)
    
    # Для правильных ответов на вопрос с учетом того, что хотя бы один человек из команды ответил
    temp_data_train = data_train.copy()
    temp_data_train['1_minus_z_pred'] = np.ones((temp_data_train.shape[0], 1)) - np.random.uniform(0, 1, (temp_data_train.shape[0], 1))
    temp_data_train_gr = temp_data_train.groupby(['tournament_id', 'team_id', 'question_num']).agg({'1_minus_z_pred':np.prod}).reset_index().rename(columns={'1_minus_z_pred':'1_minus_z_pred_multi'})
    temp_data_train = temp_data_train.merge(temp_data_train_gr, 
                                            how='left',
                                            on=['tournament_id', 'team_id', 'question_num']
                                           )
    z_pred = (z_pred / (1 - temp_data_train['1_minus_z_pred_multi'])).values
    
    # Для тех вопросов на которые команда не ответила
    z_pred[y_train.flatten()==0] = 0
    
    write_rating_to = calc_rating(prediction = z_pred, player_rating_table=write_rating_to, rating_name=rating_name)
    
    return z_pred, parameters, write_rating_to

In [19]:
pi = 0.1 # p(z==1)
n_0 = len(data_train['question_result'])-sum(data_train['question_result'])
n_u = sum(data_train['question_result'])

In [20]:
%%time
# Выведем корреляцию рейтинга команд и спрогнозированный рейтинг команд после каждого шага EM-алгоритма
N = 3
for step_number in range(N):
    
    if step_number == 0:
        # первый шаг
        z_new, coefs, player_rating = em_step(X_train=X_train.copy(), 
                                              y_train=y_train.copy(), 
                                              z_train=None,
                                              write_rating_to = player_rating,
                                              rating_name='rating_new'
                                             )
    else:
        # все остальные шаги
        z_new, coefs, player_rating = em_step(X_train=X_train.copy(), 
                                      y_train=y_train.copy(), 
                                      z_train=z_new.copy(),
                                      write_rating_to = player_rating,
                                      rating_name='rating_new'
                                     )

    
    # Проверка качества: средняя корреляция Спирмена и Кендала
    spearman, kendal = check_quality(test_df=data_test.copy(), 
                                     rating_df=player_rating.copy(), 
                                     rating_name='rating_new')
    print('Шаг', step_number+1)
    print('Корреляция Спирмана:', spearman)
    print('Корреляция Кендала:', kendal)
    print()

Шаг 1
Корреляция Спирмана: -0.715325343237144
Корреляция Кендала: -0.5528567004374216

Шаг 2
Корреляция Спирмана: -0.7258449545000236
Корреляция Кендала: -0.5619431485594815

Шаг 3
Корреляция Спирмана: -0.7274012481840572
Корреляция Кендала: -0.5639051936855034

Wall time: 3min 28s


Обновленный рейтинг игроков

In [21]:
player_rating.merge(data_train[['player_id', 'player_name', 'player_surname']].drop_duplicates(), on='player_id', how='left')[['player_id', 'rating_new', 'player_name', 'player_surname']].sort_values('rating_new', ascending=False).head(10)

,player_id,rating_new,player_name,player_surname
4201,30152,24.993722,Артём,Сорожкин
4013,28751,20.801037,Иван,Семушин
11749,110920,20.498244,Алексей,Дворянчиков
978,7008,19.081863,Алексей,Гилёв
4221,30270,18.958365,Сергей,Спешков
14741,131908,18.261247,Иван,Киселёв
3843,27499,18.055412,Наталия,Рыжанова
1724,12511,17.735158,Константин,Ильин
1698,12313,17.547570,Сергей,Иванченко
597,4270,16.569877,Александра,Брутер


# Задание 5

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

## Решение 
В качестве рейтинга сложности вопроса будем использовать средний рейтинг команд, которые не ответили на этот вопрос. Для оценки сложности турнира будем использовать средний рейтинг сложности вопросов.

In [22]:
temp_data_train = data_train.copy()
temp_data_train = temp_data_train.merge(player_rating[['player_id', 'rating_new']], how='left', on='player_id')
temp_data_train = temp_data_train.groupby(['tournament_id', 'tournament_name', 'team_id', 'question_num', 'question_result']).agg({'rating_new':'sum'}).reset_index()
temp_data_train = temp_data_train.groupby(['tournament_id', 'tournament_name']).agg({'rating_new':'mean'}).reset_index()

Топ-5 самых сложных турниров

In [23]:
temp_data_train.sort_values('rating_new', ascending=False)[['tournament_name', 'rating_new']].head(5)

,tournament_name,rating_new
452,Шестой киевский марафон. Асинхрон,57.478488
541,Чемпионат Мира. Финал. Группа А,41.886907
538,Чемпионат Мира. Этап 3. Группа А,41.299444
534,Чемпионат Мира. Этап 2. Группа А,40.252623
531,Чемпионат Мира. Этап 1. Группа А,35.415596


Топ-5 самых легких турниров

In [24]:
temp_data_train.sort_values('rating_new', ascending=True)[['tournament_name', 'rating_new']].head(5)

,tournament_name,rating_new
140,Зимник,0.124010
319,Открытый кубок Дальнего Востока (школьный зачёт),0.167535
546,Школьная лига. II тур.,0.182863
569,Открытый кубок УМЭД,0.248110
647,Открытый чемпионат Белгородской области,0.272333


Результаты рейтингования логичны: на чемпионате мира уровень сложности вопросов выше, чем на школьных турнирах